正則表達式:TVBS清理作者格式方法

In [1]:
import re
#作者欄位清理方法，要放author[0]
def tvbs_author_etl(author_text):
    author = []
    author_text = re.sub(r"記者  ", "", author_text)
    author_text = re.sub(r" / 攝影.*", "", author_text)
    author_text = re.sub(r" / ", "", author_text)
    author_text = re.sub(r"編輯  ", "", author_text)
    author_text = re.sub(r" 報導", "", author_text)
    author_text = re.sub(r" ", ",", author_text)
    #如果有多位作者，使用、斷開，並一一取出放進list
    if author_text.find(",") != -1:
        author_text = author_text.split(",")
        for a in author_text:
            author.append(a)
    elif author_text == "":
        author = []
    else:
        author.append(author_text)
    return author

正則表達式:SETN清理作者格式方法

In [2]:
#SETN作者欄位清理方法，要放author[0]
def setn_author_etl(author_text):
    author = []
    #將／XX報導XX字串清除
    author_text = re.sub(r"／.*報導.*", "", author_text)
    #將／XX特稿XX字串清除
    author_text = re.sub(r"／.*特稿.*", "", author_text)
    #將記者字串清除
    author_text = re.sub(r"記者", "", author_text)
    #將文／字串清除
    author_text = re.sub(r"文／", "", author_text)
    #如果有多位作者，使用、斷開，並一一取出放進list
    if author_text.find("、") != -1:
        author_text = author_text.split("、")
        for a in author_text:
            author.append(a)
    elif author_text == "":
        author = []
    else:
        author.append(author_text)
    return author

正則表達式:ETtoday清理作者格式方法

In [3]:
#ETtoday作者欄位清理方法，要放author[0]
def ettoday_author_etl(author_text):
    author = []
    author_text = re.sub("／.?.?報導.*", "", author_text)
    author_text = re.sub(".*記者", "", author_text)
    author_text = re.sub("\xa0", "", author_text)
    #如果有多位作者，使用、斷開，並一一取出放進list
    if author_text.find("、") != -1:
        author_text = author_text.split("、")
        for a in author_text:
            author.append(a)
    elif author_text == "":
        author = []
    else:
        author.append(author_text)
    return author

文章正反面情緒分析 需要的套件

In [4]:
#!pip install udicOpenData
#!pip install Swinger
#在dockerfile的指令
#ADD ./dict_all.txt /opt/conda/lib/python3.6/site-packages/udicOpenData/dictionary/
from udicOpenData.dictionary import *
jieba.load_userdict(os.path.join(DIR_NAME, 'dict_all.txt'))
#要等一下子
from Swinger import Swinger

Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
Loading model cost 0.849 seconds.
Prefix dict has been built succesfully.


文章正反面情緒分析方法

In [6]:
s = None
def emo_swinger(content, model='LogisticRegression'):
    global s
    if s == None:
        s = Swinger()
        s.load(model)
    # default model= "LogisticRegression", 可替換為"MultinomialNB"
    emo_result = s.swing(content)
    if emo_result == "pos":
        result = "positive"
    elif emo_result == "neg":
        result = "negative"
    return result

文章關鍵字 需要的套件

In [7]:
import jieba.analyse

文章關鍵字方法

In [8]:
def keyword_analysis(content):
    #default kewwords = 8
    kws =jieba.analyse.extract_tags(content, 5)
    return kws

文章藍綠貼標分析 需要的套件

In [9]:
#!pip install pandas 
import jieba
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.externals import joblib

將jieba斷詞寫成方法

In [10]:
def cutflow(p):
    cutresult = " ".join(jieba.cut(p))
    return cutresult.replace("\r", "").replace("\n", "")

載入詞向量模型與貝氏模型

In [11]:
clg = joblib.load('clg') 
vec = joblib.load('vec') 

/opt/conda/lib/python3.6/site-packages/sklearn/base.py:251: UserWarning: Trying to unpickle estimator MultinomialNB from version 0.20.1 when using version 0.20.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/base.py:251: UserWarning: Trying to unpickle estimator CountVectorizer from version 0.20.1 when using version 0.20.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


文章藍綠貼標分析方法:貝氏定理

In [12]:
def label_predict(text):
    docs_news = cutflow(text)
    c=[{"content" : docs_news,
       "tag" : 0}]
    d = pd.DataFrame(c)
    test_counts = vec.transform(d["content"])
    pre = clg.predict(test_counts)
    label_result = list(pre)[0]
    if label_result == '1':
        result = "blue"
    elif label_result == '0':
        result = "green"
    return result

文章摘要分析 需要的套件

In [13]:
#!pip install networkx
#!pip install textrank4zh
from  __future__  import print_function
import jieba
import numpy
import networkx
import sys
import codecs 
from textrank4zh import TextRank4Keyword ,TextRank4Sentence

文章摘要方法:TextRank

In [14]:
def news_summary(text):
    tr4s = TextRank4Sentence()
    tr4s.analyze( text = text, lower = True , source = ' all_filters ' )
    for item in tr4s.get_key_sentences( num = 1 ):
        return item.sentence

Elasticsearch 需要的套件

In [15]:
#!pip install elasticsearch
from elasticsearch import Elasticsearch

In [15]:
# 建立Elasticsearch連線
es = Elasticsearch(['elasticsearch:9200'])

In [16]:
# 在Elasticsearch建立index，亦可加入body參數先定義mapping格式
es.indices.create(index = 'news', ignore = 400)

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'news'}

In [17]:
# 將Elasticsearch定義所放入的資料格式:mapping
es.indices.put_mapping(index = "news",
                       doc_type = "politics",                
                       body = {
                        "properties": {
                            "source": {"type": "text"},
                            "url": {"type": "text"},
                            "title": {"type": "text"},
                            "date_": {"type": "text"},
                            "author": {"type": "text"},
                            "content": {"type": "text"},
                            "kw": {"type": "text"},
                            "img_url": {"type": "text"},
                            "@timestamp": {"type": "date"}
                        }
                    }
                )

{'acknowledged': True}

In [16]:
# 為了將匯入資料的時間加入時區
from datetime import datetime, timedelta, timezone
tz_utc_8 = timezone(timedelta(hours=8)) # 創建時區UTC+8:00

傳送log 需要的套件

In [17]:
#!pip install python-logstash
import logging
import logstash
import sys

In [18]:
host = "logstash"
# 資料清理與分析log
etl_analysis_logger = logging.getLogger('etl_analysis_logger')
etl_analysis_logger.setLevel(logging.INFO)
# TCP
etl_analysis_logger.addHandler(logstash.TCPLogstashHandler(host, 5000, version=1))

# 資料傳輸到MySQL log
mysql_logger = logging.getLogger('mysql_logger')
mysql_logger.setLevel(logging.INFO)
# TCP
mysql_logger.addHandler(logstash.TCPLogstashHandler(host, 5000, version=1))

# 資料傳輸到Elasticsearch log
elasticsearch_logger = logging.getLogger('elasticsearch_logger')
elasticsearch_logger.setLevel(logging.INFO)
# TCP
elasticsearch_logger.addHandler(logstash.TCPLogstashHandler(host, 5000, version=1))

Kafka Consumer需要的套件

In [19]:
#!pip install kafka
from kafka import KafkaConsumer, TopicPartition
import sys
import json, time, requests

In [20]:
if __name__ == "__main__":
    # 步驟1.設定要連線到Kafka集群的相關設定, 產生一個Kafka的Consumer的實例
    consumer = KafkaConsumer(
        # Kafka集群在那裡?
        bootstrap_servers=["kafka1:29092"],
        # ConsumerGroup的名稱
        group_id="politics_news01",
        # 指定msgKey的反序列化器, 若Key為None, 無法反序列化
        key_deserializer=bytes.decode,
        # 指定msgValue的反序列化器
        #value_deserializer=bytes.decode,
        value_deserializer=lambda m: json.loads(m.decode('utf-8')),
        # 是否從這個ConsumerGroup尚未讀取的partition / offset開始讀
        auto_offset_reset="earliest",
    )
    # 步驟2.指定想要訂閱訊息的topic名稱
    topic_name = "politics_news"
    # 步驟3.讓Consumer向Kafka集群訂閱指定的topic
    consumer.subscribe(topics = topic_name)


    # 步驟4.持續的拉取Kafka有進來的訊息
    try:
        print("Now listening for incoming messages ...","\n")
        # 持續監控是否有新的record進來 - 方法一
        for record in consumer:
            topic = record.topic
            partition = record.partition
            offset = record.offset
            timestamp = record.timestamp
        
            # 取出msgKey與msgValue
            msgKey = record.key
            msgValue = record.value
            try:
                # 將新聞作者欄位做處理後再放回
                if msgValue["source"] == "TVBS":      
                    msgValue["author"] = tvbs_author_etl(msgValue["author"][0])
                elif msgValue["source"] == "SETN":
                    msgValue["author"] = setn_author_etl(msgValue["author"][0])
                elif msgValue["source"] == "ETtoday":
                    msgValue["author"] = ettoday_author_etl(msgValue["author"][0])

                # 將新聞內文做文章情緒分析並加入欄位與結果
                msgValue["ariticle_emotion"] = emo_swinger(msgValue["content"])
                # 將新聞內文做關鍵字分析再合併至新聞抓到的關鍵字列表，並把重複的篩選掉
                content_keyword = keyword_analysis(msgValue["content"])
                for k in content_keyword:
                    msgValue["kw"].append(k)
                msgValue["kw"] = list(set(msgValue["kw"]))
                # 將新聞內文做藍綠分析並加入欄位結果
                msgValue["label"] = label_predict(msgValue["content"])
                # 將新聞做文章摘要並加入欄位與結果
                msgValue["abstract"] = news_summary(msgValue["content"])

                # 秀出metadata與msgKey & msgValue訊息
                print("topic=%s, partition=%s, offset=%s : (key=%s, value=%s)" % (record.topic, record.partition,
                                                                                  record.offset, record.key, msgValue))
                print("\n")             
                
                # TCP連線到logstash
                etl_analysis_logger.addHandler(logstash.TCPLogstashHandler(host, 5000, version=1))
                # 爬蟲成功傳送至Elasticsearch的log訊息
                etl_analysis_logger.info('python-etl-logstash: Success!')
                etl_analysis_logger.handlers.clear()
            except:
                print("Data ETL was unsuccessful!!")
                etl_analysis_logger.addHandler(logstash.TCPLogstashHandler(host, 5000, version=1))
                etl_analysis_logger.error('python-etl-logstash: Error message!! [' + msgValue["url"] + ']')
                etl_analysis.handlers.clear()

            
            try:
                print("Now sending data to MySQL!")
                # 將資料送入MySQL
                ip_location = 'chatbot_api'

                News = msgValue

                # 將json傳回API Server
                Endpoint = 'http://%s:5000/news' % (ip_location)

                # header要特別註明是json格式
                Header = {'Content-Type':'application/json'}

                # 傳送post對API server新增資料 
                Response = requests.post(Endpoint,headers=Header,data=json.dumps(News))

                # 印出Response的資料訊息
                print(Response)
                Response = Response.json()
                print(Response)
                print("Data was sent to MySQL successfully!")
                print("\n")
                
                # TCP連線到logstash
                mysql_logger.addHandler(logstash.TCPLogstashHandler(host, 5000, version=1))
                # 爬蟲成功傳送至MySQL的log訊息
                mysql_logger.info('python-data2mysql-logstash: Success!')
                mysql_logger.handlers.clear()
            except:
                print("Error message : Data was sent to MySQL unsuccessfully!!")
                mysql_logger.addHandler(logstash.TCPLogstashHandler(host, 5000, version=1))
                mysql_logger.error('python-data2mysql-logstash: Error message!! [' + News["url"] + ']')
                mysql_logger.handlers.clear()
                
            try:
                print("Now sending data to elasticsearch!")
                # 將資料送入Elasticsearch
                elasticsearch_data = msgValue
                elasticsearch_data["@timestamp"] = datetime.now().replace(tzinfo = tz_utc_8)            
                # 將資料放入Elasticsearch，若沒有index，會直接建立index、type 
                es.index(index = 'news', doc_type = 'politics', body = elasticsearch_data, refresh = True )
                print("Data was sent to elasticsearch successfully!")
                print("\n")
                
                # TCP連線到logstash
                elasticsearch_logger.addHandler(logstash.TCPLogstashHandler(host, 5000, version=1))
                # 爬蟲成功傳送至Elasticsearch的log訊息
                elasticsearch_logger.info('python-data2elasticsearch-logstash: Success!')
                elasticsearch_logger.handlers.clear()     
            except:
                print("Error message: Data was sent to elasticsearch unsuccessfully!!")
                elasticsearch_logger.addHandler(logstash.TCPLogstashHandler(host, 5000, version=1))
                elasticsearch_logger.error('python-data2elasticsearch-logstash: Error message!! [' + elasticsearch_data["url"] + ']')
                elasticsearch_logger.handlers.clear()

    except:
        # 錯誤處理
        e_type, e_value, e_traceback = sys.exc_info()
        print("type ==> %s" % (e_type))
        print("value ==> %s" % (e_value))
        print("traceback ==> file name: %s" % (e_traceback.tb_frame.f_code.co_filename))
        print("traceback ==> line no: %s" % (e_traceback.tb_lineno))
        print("traceback ==> function name: %s" % (e_traceback.tb_frame.f_code.co_name))
    finally:
        consumer.close()

Now listening for incoming messages ... 

load default bestMainFeatures
load default bestMainFeatures success!!
load model from LogisticRegression


/opt/conda/lib/python3.6/site-packages/sklearn/base.py:251: UserWarning: Trying to unpickle estimator DictVectorizer from version 0.18.1 when using version 0.20.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/base.py:251: UserWarning: Trying to unpickle estimator LabelEncoder from version 0.18.1 when using version 0.20.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/base.py:251: UserWarning: Trying to unpickle estimator LogisticRegression from version 0.18.1 when using version 0.20.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


topic=politics_news, partition=0, offset=0 : (key=ettoday, value={'source': 'ETtoday', 'url': 'https://www.ettoday.net/news/20181229/1343337.htm', 'title': '國民黨要培養年輕人 連勝文：個人背景不重要', 'date_': '2018/12/29 00:06', 'author': ['政治中心'], 'content': '九合一選雖然結束，儘管國民黨大勝，但世代交替、中生代接班問題也再度浮出，甚至將國民黨目前世代交替困境的責任，指向前總統馬英九。國民黨中央委員連勝文28日就發文說，他一直認為個人背景好不好，學歷高或低，並不重要，最重要的是「個人表現」，國民黨必須要建立「performance-based」的長期人才選用方式，至少他就是這樣在培養年輕人。連勝文表示，國民黨世代交替已經喊了超過十年了，但執政時不做，等到在野時就變得更困難了；因為在上位者已無資源，也沒有足夠的位置，而在競爭者眾的情況下，自私的人就會把僅存的位置做為自己拚大位的政治資本，而沽名釣譽者，則會執著於「用人不分藍綠」的迷思，找一堆跟國民黨理念價值完全不同的人，變成個人衞隊，但國民黨一垮，這些人也就跑了！這樣下來，黨內人才，即便表現好，辛苦付出，也都意興闌珊了。連勝文指出，他一直認為個人背景好不好，學歷高或低，並不重要，最重要的是「個人表現」！國民黨必須要建立「performance-based」的長期人才選用方式，以長期個人表現為基礎，給有「實質付出」的人表現機會，不因一兩次挫折而中斷，也不能只靠少數樣板，更不能靠拍馬屁來決定誰有機會，至少他就是這樣在培養年輕人。連勝文說，必須承認國民黨內，像蔣乃辛或之前的蔡正元這種願意主動讓位的人並不多，所以他主張，讓65歲以上，長期表現傑出，同時還有意願服務的資深區域立委，升做不分區立委，建立國會資深議員制度，而讓青壯世代有意願者，去競選區域立委，這樣才能兼顧經驗與衝勁，發揮最大的戰力。「國民黨不缺少數樣板，也不缺各種逢迎溢美之聲」，連勝文說，唯一缺的就是有能力改變現狀的人，可以大公無私，為國民黨的未來，真正樹立一套典範！希望我們在數年後，不會再看到類似的文章，只是主角從馬英九換成别人。', 'kw':

topic=politics_news, partition=0, offset=6 : (key=ettoday, value={'source': 'ETtoday', 'url': 'https://www.ettoday.net/news/20181228/1343033.htm', 'title': '砲打蔡英文 王金平：領導人一定要讓朝野對立削減', 'date_': '2018/12/28 17:14', 'author': ['賴于榛'], 'content': '前立法院長、國民黨立委王金平28日砲打總統蔡英文，他說，台灣目前面臨許多問題，國內充滿勞資對立、族群對立、世代對立，問題越來越混亂，且藍綠互相抵制，所以「當然身為國家領導人一定要讓朝野對立削減，藍綠對立化解，跨越藍綠共謀國是」。王金平28日赴銘傳大學演講「國會秘辛與政治傳播」，從兩岸、國內情勢與貿易解析台灣問題，兩岸部分他表示，九二共識很多人解讀不一樣，但目前大陸就是認為說執政黨沒有承認九二共識，因此不多所往來，且開始對於台灣各方面給予不滿意的政策、各方面的措施，加上台灣也是去中國化去得很厲害，是兩岸緊張的原因，他建議政府要想辦法改善兩岸關係，坐下來磋商不要阻礙台灣發展，否則累積下去會有問題。\n\n國內議題部分，王金平說，國內充滿勞資對立、族群對立、世代對立，問題越來越混亂，且藍綠互相抵制，提醒「當然身為國家領導人一定要讓朝野對立削減，藍綠對立化解，跨越藍綠共謀國是」，不能各自堅持、各談各的，否則這個國家怎麼辦。經濟困境部分，王金平說五缺要解決也不是那麼容易，且何況台灣沒有關稅彼此的公平對待，想誰想來台灣投資，沒有投資就業的機會就少了，加上CPTPP被因為反核食公投被日本阻撓，以及兩岸FTA也有阻力。王金平認為經濟發展因為兩岸沒有通力合作，造成經濟發展阻礙，他提醒蔡英文一定要把經濟搞好，且對兩岸問題要負起絕對責任，讓國家安全、兩岸安定、社會安心。會後媒體追問朱立倫、吳敦義與王金平誰當領導人比較能化解藍綠對立，王金平則表示，每個人都有努力空間，只要努力都有機會。', 'kw': ['蔡英文', '國民黨', '對立', '藍綠', '台灣', '兩岸', '王金平', '2020'], 'img_url': 'https://cdn2.ettoday.net/images/3796/d37

topic=politics_news, partition=0, offset=12 : (key=ettoday, value={'source': 'ETtoday', 'url': 'https://www.ettoday.net/news/20181228/1342746.htm', 'title': '蘇貞昌「穿越時空」自打臉 回鍋行政院因一張照片被嗆爆', 'date_': '2018/12/28 12:34', 'author': ['政治中心'], 'content': '前行政院長蘇貞長傳出將回鍋，接棒閣揆賴清德。不過，蘇貞昌3個月前接受媒體專訪時，曾稱「不會再入閣」，這次「穿越時空」自打臉，引發熱議。民進黨九合一大選慘敗，賴清德隨即請辭負責遭慰留，但他辭意堅定，日前就曾表示「時間一到會離開」，而今蘇貞昌被爆將重掌行政院，最快1月總預算案後就會上任。據了解，行政院秘書長卓榮泰參選黨主席後，黨內已經開始啟動布局，包括部分縣市一級主管在內的蘇的子弟兵，也紛紛請辭或未上任，日前已在政壇傳得沸沸揚揚，蘇內閣上任已經呼之欲出。不過，當初接受民進黨徵召參選新北市長的蘇貞昌，曾在9月接受媒體專訪時，對於未來若競選失利是否將到行政院「服務更多民眾」，他堅定的說，「我都組閣過了，不會再入閣了！」沒想到今卻傳出「賴下蘇上」，引來網友狂酸，「正都騙過神明了，再騙人一次也沒差」、「原來選輸就有行政院長可當」、「連敗選都能升官的候選人真令人羨慕」。對於蘇貞昌任行政院長，府院未多做回應，強調沒有相關訊息。蘇貞昌陣營則重申，尊重賴院長的想法。', 'kw': ['上任', '行政院長', '民進黨', '賴清德', '蘇貞昌', '行政院'], 'img_url': 'https://cdn2.ettoday.net/images/3713/d3713882.jpg', 'ariticle_emotion': 'negative', 'label': 'green', 'abstract': '不過，當初接受民進黨徵召參選新北市長的蘇貞昌，曾在9月接受媒體專訪時，對於未來若競選失利是否將到行政院「服務更多民眾」，他堅定的說，「我都組閣過了，不會再入閣了'})


Now sending data to MySQL!
Error message : Data was sent 

...

topic=politics_news, partition=0, offset=25 : (key=ettoday, value={'source': 'ETtoday', 'url': 'https://www.ettoday.net/news/20181227/1342324.htm', 'title': '高嘉瑜「眼球運動」春聯加碼1萬張秒殺了！ 網友哀嚎：快出Line貼圖', 'date_': '2018/12/27 22:26', 'author': ['政治中心'], 'content': '被封為港湖女神的台北市議員高嘉瑜，27日在臉書宣布推出「眼球運動」春聯，吸引粉絲瘋搶後她再加碼1萬張，結果不到3個小時8千張春聯已經秒殺，搶不到春聯的網友留言要高嘉瑜「快出Line貼圖」。高嘉瑜推出「抱豬甜笑款」及「眼球運動款」春聯，提供給想要的粉絲索取，她也貼心提醒網友，數量有限很可能會秒殺，沒想到一個小時過後，春聯真的被搶光了，於是她再加碼到1萬份，不到3個小時，吸引2千粉絲瘋搶8千張春聯。高嘉瑜開心在臉書說，謝謝大家喜歡，也提醒大家春聯印好會在公告，不提供寄送服務，要到服務處親領。搶不到春聯的網友留言表示，「快點出賴貼圖」、「這太秒殺了吧～連車尾燈都看不到」、「我沒登記到」、「果然是港湖最高票」。', 'kw': ['秒殺', '粉絲', '小時', '春聯', '高嘉瑜'], 'img_url': 'https://cdn2.ettoday.net/images/3783/d3783932.jpg', 'ariticle_emotion': 'positive', 'label': 'green', 'abstract': '高嘉瑜推出「抱豬甜笑款」及「眼球運動款」春聯，提供給想要的粉絲索取，她也貼心提醒網友，數量有限很可能會秒殺，沒想到一個小時過後，春聯真的被搶光了，於是她再加碼到1萬份，不到3個小時，吸引2千粉絲瘋搶8千張春聯'})


Now sending data to MySQL!
Error message : Data was sent to MySQL unsuccessfully!!
Now sending data to elasticsearch!
Error message: Data was sent to elasticsea

topic=politics_news, partition=0, offset=33 : (key=ettoday, value={'source': 'ETtoday', 'url': 'https://www.ettoday.net/news/20181227/1341972.htm', 'title': '影／學姐黃瀞瑩被「熊抱」 網友崩潰喊：放開那個女孩！', 'date_': '2018/12/27 16:05', 'author': ['政治中心'], 'content': '台北市長柯文哲7月參與網路節目錄製「一日幕僚」，意外捧紅了長相清秀的幕僚黃瀞瑩，網友封她為「學姐」，擁有超高人氣的黃瀞瑩27日在臉書PO出一張被熊讚擁抱的照片，幽默寫下「被熊抱了啦」，「#首度合作」、「#熊讚是個專業的前輩」，引起大批網友崩潰喊，「放開那個女孩！」網友回說，「裡面是阿公還是陳百祥，扇子拿來了啦～」、「裡面的工讀生給我出來 扇子拿來」、「請教熊肉火鍋的料理方法」、「怎麼看都是邰哥在裡面」、「放開那個女孩」、「明年的今天是熊讚的祭日」、「工讀生福利這麼好哦XD」小牛柯昱安也留言「裡面是局長(誤)」，引來網友說「這樣陷害大熊長官好嗎？」、「大熊快出來」、「大熊要被通緝了」、「有人洩密了」、「這時候要請局長提出不在場證明了」，還有人說「確定不是市長？」、「 我覺得不是！因為局長穿不下....」，討論串也吸引大熊劉奕霆tag小牛回應「柯昱安............」，透露出他的無奈。台北市長柯文哲24日公布小內閣名單，由於發言人劉奕霆接任觀傳局長，因此副發言人部分由「學姐」黃瀞瑩、「小牛」柯昱安共同接任，替劉奕霆分擔發言人的工作。', 'kw': ['大熊', '學姐', '黃瀞瑩', '裡面', '柯昱安', '熊讚', '網友'], 'img_url': 'https://cdn2.ettoday.net/images/3793/d3793527.jpg', 'ariticle_emotion': 'negative', 'label': 'green', 'abstract': '」網友回說，「裡面是阿公還是陳百祥，扇子拿來了啦～」、「裡面的工讀生給我出來 扇子拿來」、「請教熊肉火鍋的料理方法」、「怎麼看都是邰哥在裡面」、「放開那個女孩」、「明年的今天是熊讚的祭日」、

topic=politics_news, partition=0, offset=40 : (key=ettoday, value={'source': 'ETtoday', 'url': 'https://www.ettoday.net/news/20181227/1341865.htm', 'title': '悼江丙坤 馬英九：對兩岸關係貢獻史上少有、全國罕見', 'date_': '2018/12/27 14:13', 'author': ['政治中心'], 'content': '前總統馬英九27日表示，前海基會董事長江丙坤臨走前，他趕到加護病房，向江丙坤低聲告別，並肯定對國家、尤其是對兩岸關係的重大貢獻；他26日到三芝送江丙坤最後一程，並為江丙坤覆上國旗，「我內心有著深沈的悲痛與懷念」馬英九在臉書提及昔日與江丙坤共戰情懷。民國82年，江丙坤與馬英九分別擔任經濟部長及法務部長，在隨後3年中，不論在行政院院會或立法院院會，都比鄰而坐，讓他有很多機會瞭解江丙坤的為人處事。馬英九指出，江丙坤比自己大18歲，是台中高農苦學出身，省立法商學院畢業，並獲得國民黨第一屆中山獎學金留學日本，取得東京大學農業經濟學博士；同時也精通英、日文，熟悉經貿與外交事務，做人誠懇謙和、做事認真幹練，長官及同事們都很佩服。馬英九表示，民國97年他擔任總統後，特別邀請擔任過經濟部長、經建會主委、與立法院副院長的江丙坤，委屈出任海基會董事長一職，但江丙坤把這個職位發揮得淋漓盡致，任內簽下18個兩岸協議，使兩岸關係真正進入和平繁榮時代，是60多年來僅見，嘉惠無數兩岸人民及企業。馬英九指出，海基會籌建新大樓時，江丙坤自行募款，沒讓政府出錢，對兩岸關係付出的心血與貢獻，史上少有，全國罕見，大家會永遠記得江丙坤的努力與成就；他26日到三芝送江丙坤最後一程，並在靈柩上覆上國旗，代表國家對江丙坤的肯定，也是最好的見證，「永別了，丙坤兄，望您一路好走。」', 'kw': ['海基會', '擔任', '兩岸關係', '馬英九', '江丙坤'], 'img_url': 'https://cdn2.ettoday.net/images/3793/d3793164.jpg', 'ariticle_emotion': 'negative', 'label': 'blue', 'abstract': '馬英九表示，民國

topic=politics_news, partition=0, offset=48 : (key=ettoday, value={'source': 'ETtoday', 'url': 'https://www.ettoday.net/news/20181227/1341555.htm', 'title': '民進黨砍完吳音寧、葉俊榮 唐湘龍：下一個輪到謝長廷', 'date_': '2018/12/27 11:23', 'author': ['政治中心'], 'content': '民進黨在2018九合一大選中大敗，黨內出現檢討聲浪，第一位下台的就是被網友稱為「高年薪實習生」北農總經理吳音寧，接著是因為管中閔案遞辭呈的教育部長葉俊榮。資深媒體人唐湘龍認為，按照民進黨這樣一步一步「災後重建」下去，下一個「斬」的就是駐日代表謝長廷。唐湘龍在25日播出的《少康戰情室》中提到，民進黨現在開始在做一些「災後重建」、「災後清理」，最早的是吳音寧，大家一開始會覺得，為什麼把帳算在吳音寧身上？現在就知道，吳音寧只是一個開始，是很小很小的一點。現在連葉俊榮都可以請辭，把「卡管」變成「卡賴」，雖然獨派會因此憤怒，但是願意在這種情況下設停損點，願意在這種情況下，像台北事長柯文哲講的「認賠殺出」，都是好的！唐湘龍說，「現在謝長廷看起來是下一個，我認為謝長廷會聞到排隊準備被問斬的味道，下一個大概就是斬謝長廷，謝長廷雖然在簡訊發了『了解』兩個字，但以他的政治歷練，應該知道自己的政治生命大概只到這一刻了。」蘇啓誠輕生爭議延燒，遺孀受訪時透露，丈夫並非因為假新聞的壓力而死，而是開檢討會前一天接到外交部電話，被告知調職、懲處，以及駐處人員考績丙等，而丈夫的遺書是以「羞辱」兩個字來描述要受到的懲處。聲明一出，引發外界認為「外交部逼死蘇啓誠」。國民黨立院黨團總召江啟臣對此批評，謝長廷常常回國處理和外交無關的事情，可以說是最不務正業的駐日代表。而謝長廷面對批評，僅回覆「了解」二字，未多做其他回應。', 'kw': ['現在', '吳音寧', '謝長廷', '葉俊榮', '日本', '民進黨', '駐日代表', '一個', '唐湘龍', '蘇啓誠'], 'img_url': 'https://cdn2.ettoday.net/images/3717/d3717364.jpg', 'ariticle_e

topic=politics_news, partition=0, offset=55 : (key=ettoday, value={'source': 'ETtoday', 'url': 'https://www.ettoday.net/news/20181227/1341193.htm', 'title': '政治獻金大公開！柯文哲上次選市長收1.4億 7成8來自個人捐贈', 'date_': '2018/12/27 09:31', 'author': ['潘永鴻'], 'content': '監察院26日公布最新《廉政專刊》，台北市長柯文哲在今年10月 25 日申請更正申報2014年台北市長選舉的政治獻金，就上次台北市長選舉，柯文哲共花費 1.4億元左右、收入則是1.39億元，總餘額是負2,537,103元，等於不足253多萬元。監察院公布第139期廉政專刊。柯文哲的政治獻金收支結算表裡，103年4月25日到12月29日，總收入共139,825,881元，這包括，個人捐贈收入115,007,009元、營利事業捐贈收入8,643,400元、政黨捐贈收入0元、人民團體捐贈收入1,807,000元、匿名捐贈收入14,346,822元、其他收入21,650元。超過三萬元之收入合計83,656,141元，金錢收入總額：138,742,341元、非金錢收入總額：1,083,540元。就支出部分總計：142,362,984元，其中較大宗的支出包括，人事費用支出33,340,335元、宣傳支出64,926,802元、集會支出26,694,072元。整體而言，總餘額是負2,537,103元。', 'kw': ['政治獻金', '收入', '監察院', '柯文哲', '捐贈', '支出', '26'], 'img_url': 'https://cdn2.ettoday.net/images/3789/d3789565.jpg', 'ariticle_emotion': 'negative', 'label': 'blue', 'abstract': '柯文哲的政治獻金收支結算表裡，103年4月25日到12月29日，總收入共139,825,881元，這包括，個人捐贈收入115,007,009元、營利事業捐贈收入8,643,400元、政黨捐贈收入0元、人民團體捐贈收入1,807,00